In [1]:
import pandas as pd

In [2]:
dataset=pd.read_csv("balanced_dataset.csv")

In [3]:
dataset

,app_name,response_time,error_rate,cpu_usage,memory_usage,disk_space,active_users,downtime
0,NIP,21,77,31,41,51,11000,0
1,NIP,21,1,31,41,51,11000,0
2,NIP,18,65,28,38,48,8000,0
3,NIP,17,85,27,37,47,7000,0
4,NIP,14,18,24,34,44,4000,0
...,...,...,...,...,...,...,...,...
1163,NIP,10,54,20,30,40,0,1
1164,NIP,14,31,24,34,44,4000,1
1165,NIP,18,95,28,38,48,8000,1
1166,NIP,22,75,32,42,52,12000,1


In [4]:
dataset=pd.get_dummies(dataset,dtype=int,drop_first=True)

In [5]:
dataset

,response_time,error_rate,cpu_usage,memory_usage,disk_space,active_users,downtime
0,21,77,31,41,51,11000,0
1,21,1,31,41,51,11000,0
2,18,65,28,38,48,8000,0
3,17,85,27,37,47,7000,0
4,14,18,24,34,44,4000,0
...,...,...,...,...,...,...,...
1163,10,54,20,30,40,0,1
1164,14,31,24,34,44,4000,1
1165,18,95,28,38,48,8000,1
1166,22,75,32,42,52,12000,1


In [6]:
dataset.columns

Index(['response_time', 'error_rate', 'cpu_usage', 'memory_usage',
       'disk_space', 'active_users', 'downtime'],
      dtype='object')

In [7]:
indep=dataset[['response_time', 'error_rate', 'cpu_usage', 'memory_usage','disk_space', 'active_users']]

In [8]:
dep=dataset[['downtime']]

In [9]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split( indep, dep, test_size=0.30, random_state=0)

In [10]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
X_train=scaler.fit_transform(x_train)
X_test=scaler.transform(x_test)

In [11]:
from sklearn.neighbors import KNeighborsClassifier

In [12]:
from sklearn.model_selection import GridSearchCV
param_grid = {'metric':['minkowski','callable'],
              'algorithm':['auto', 'ball_tree', 'kd_tree', 'brute'],
              'weights':['uniform','distance']}
grid = GridSearchCV(KNeighborsClassifier(), param_grid, refit = True, verbose = 3,n_jobs=-1,scoring='f1_weighted')
grid.fit(X_train,y_train)

Fitting 5 folds for each of 16 candidates, totalling 80 fits


C:\Anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:547: FitFailedWarning: 
40 fits failed out of a total of 80.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
4 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Anaconda3\Lib\site-packages\sklearn\base.py", line 1467, in wrapper
    estimator._validate_params()
  File "C:\Anaconda3\Lib\site-packages\sklearn\base.py", line 666, in _validate_params
    validate_parameter_constraints(
  File "C:\Anaconda3\Lib\site-packages\sklearn\utils\_param_validation.py", line 95, in validate_par

GridSearchCV(estimator=KNeighborsClassifier(), n_jobs=-1,
             param_grid={'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
                         'metric': ['minkowski', 'callable'],
                         'weights': ['uniform', 'distance']},
             scoring='f1_weighted', verbose=3)

In [13]:
re=grid.cv_results_
grid_predictions = grid.predict(X_test)
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_test,grid_predictions)
from sklearn.metrics import classification_report
clf_report = classification_report(y_test, grid_predictions)

In [14]:
from sklearn.metrics import f1_score
f1_macro=f1_score(y_test,grid_predictions,average='weighted')
print("the f1_macro value for best parameter{}:".format(grid.best_params_),f1_macro)

the f1_macro value for best parameter{'algorithm': 'auto', 'metric': 'minkowski', 'weights': 'uniform'}: 0.8376083470123205


In [15]:
print("the confusion matrix:\n",cm)

the confusion matrix:
 [[123  50]
 [  6 172]]


In [16]:
print("the report:\n",clf_report)

the report:
               precision    recall  f1-score   support

           0       0.95      0.71      0.81       173
           1       0.77      0.97      0.86       178

    accuracy                           0.84       351
   macro avg       0.86      0.84      0.84       351
weighted avg       0.86      0.84      0.84       351



In [17]:
import pickle

In [18]:
filename="finalized_model_knn.pkl"

In [19]:
filename1="finalized_model_sc.pkl"

In [20]:
pickle.dump(grid,open(filename,"wb"))

In [21]:
pickle.dump(scaler,open(filename1,"wb"))

In [22]:
loaded_model=pickle.load(open("finalized_model_knn.pkl","rb"))

In [23]:
loaded_model1=pickle.load(open("finalized_model_sc.pkl","rb"))

In [24]:
preinput=scaler.transform([[12,20,30,50,37,12000]])
preinput
#'response_time', 'error_rate', 'cpu_usage', 'memory_usage','disk_space', 'active_users'

C:\Anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


array([[-1.1060651 , -1.11792516,  1.07865581,  3.80955694, -2.47151566,
         1.62483604]])

In [25]:
result=loaded_model.predict(preinput)
result

array([0], dtype=int64)